## PICRUST 
Input is the three files after denoising: metadata.tsv, repseqs.fna, and table.biom
First thing to do is to export the repseq and table into recognizeable format


In [1]:
import os
import pandas as pd
path = '../data/qiime2/filtered/'
i_path = os.path.join(path, 'exported-feature-table')
os.listdir(i_path)

['.ipynb_checkpoints',
 'taxonomy.tsv',
 'taxonomy_silva-138-99-nb-classifier.tsv',
 'taxonomy_gg-13-8-99-nb-classifier.tsv',
 'taxonomy_gg-13-8-99-515-806-nb-classifier.tsv',
 'dna-sequences.fasta',
 'feature-table.tsv',
 'feature-table.biom']

In [2]:
! biom head -i {i_path}/feature-table.biom
! biom summarize-table -i {i_path}/feature-table.biom

# Constructed from biom file
#OTU ID	P8-rep4	P5-rep4	P9-rep1	P9-rep5	P9-rep4
e5400356daabbc5a41f935af70513043	1598.0	630.0	297.0	795.0	1449.0
7cbebea20e305a557ffc3dc23bae61ae	1111.0	36.0	127.0	335.0	610.0
4dc5023fdc00f325e169fced16dca21c	1022.0	93.0	323.0	863.0	2013.0
4c076cffb4dc7aaae47cb237d7067066	739.0	942.0	104.0	347.0	814.0
3c4c98cf9b1264b89f9ecd0812a0f7d8	632.0	195.0	224.0	571.0	1259.0
Num samples: 15
Num observations: 8,820
Total count: 685,458
Table density (fraction of non-zero values): 0.337

Counts/sample summary:
 Min: 7,212.000
 Max: 119,925.000
 Median: 40,711.000
 Mean: 45,697.200
 Std. dev.: 29,845.346
 Sample Metadata Categories: None provided
 Observation Metadata Categories: None provided

Counts/sample detail:
P8-rep1: 7,212.000
P9-rep2: 13,593.000
P5-rep3: 18,002.000
P5-rep1: 25,899.000
P9-rep1: 26,396.000
P5-rep5: 34,762.000
P8-rep4: 39,851.000
P8-rep3: 40,711.000
P8-rep5: 42,000.000
P5-rep4: 43,658.000
P5-rep2: 48,907.000
P9-rep5: 61,270.000
P8-rep2: 61,943.000


In [3]:
! place_seqs.py -s {path}/exported-feature-table/dna-sequences.fasta \
    -o ../data/picrust2/01-out.tre \
    -p 8 \
    --intermediate ../data/picrust2/01-intermediate/place_seqs \
    --verbose

hmmalign --trim --dna --mapali /home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/lib/python3.6/site-packages/picrust2/default_files/prokaryotic/pro_ref/pro_ref.fna.gz --informat FASTA -o ../data/picrust2/01-intermediate/place_seqs/query_align.stockholm /home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/lib/python3.6/site-packages/picrust2/default_files/prokaryotic/pro_ref/pro_ref.hmm ../data/qiime2/filtered//exported-feature-table/dna-sequences.fasta


All raw input sequences were the same length (400)

epa-ng --tree /home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/lib/python3.6/site-packages/picrust2/default_files/prokaryotic/pro_ref/pro_ref.tre --ref-msa ../data/picrust2/01-intermediate/place_seqs/ref_seqs_hmmalign.fasta --query ../data/picrust2/01-intermediate/place_seqs/study_seqs_hmmalign.fasta --chunk-size 5000 -T 8 -m /home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/lib/python3.6/site-packages/picrust2/default_files/prokaryotic/pro_ref/pro_ref.model -w ../data/picrust2/01-inter

In [4]:
! hsp.py \
    -i 16S \
    -t ../data/picrust2/01-out.tre \
    -o ../data/picrust2/02-marker_predicted_and_nsti.tsv.gz \
    -p 8 -n \
    --verbose

! hsp.py \
    -i EC \
    -t ../data/picrust2/01-out.tre \
    -o ../data/picrust2/02-EC_predicted.tsv.gz \
    -p 8 \
    --verbose

Rscript /home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/lib/python3.6/site-packages/picrust2/Rscripts/castor_nsti.R ../data/picrust2/01-out.tre /tmp/tmpuiar1fq0/known_tips.txt /tmp/tmpuiar1fq0/nsti_out.txt


Rscript /home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/lib/python3.6/site-packages/picrust2/Rscripts/castor_hsp.R ../data/picrust2/01-out.tre /tmp/tmphysr9cyp/subset_tab_0 mp FALSE FALSE /tmp/tmp0f3t_adq/predicted_counts.txt /tmp/tmp0f3t_adq/predicted_ci.txt 100


Rscript /home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/lib/python3.6/site-packages/picrust2/Rscripts/castor_hsp.R ../data/picrust2/01-out.tre /tmp/tmpinw307sa/subset_tab_0 mp FALSE FALSE /tmp/tmpr_fwp7yb/predicted_counts.txt /tmp/tmpr_fwp7yb/predicted_ci.txt 100
Rscript /home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/lib/python3.6/site-packages/picrust2/Rscripts/castor_hsp.R ../data/picrust2/01-out.tre /tmp/tmpinw307sa/subset_tab_2 mp FALSE FALSE /tmp/tmpz6hvxmy6/predicted_counts.txt /tmp/tmpz6hvxmy6/predicted_c

In [5]:
#! zless -S ../data/picrust2/02-marker_predicted_and_nsti.tsv.gz
#! zless -S ../data/picrust2/02-EC_predicted.tsv.gz

In [6]:
! metagenome_pipeline.py \
    -i {i_path}/feature-table.biom \
    -m ../data/picrust2/02-marker_predicted_and_nsti.tsv.gz \
    -f ../data/picrust2/02-EC_predicted.tsv.gz \
    -o ../data/picrust2/03-EC_metagenome_out \
    --strat_out

80 of 8820 ASVs were above the max NSTI cut-off of 2.0 and were removed.


In [7]:
# ! zless -S ../data/picrust2/03-EC_metagenome_out/pred_metagenome_contrib.tsv.gz

In [13]:
! pathway_pipeline.py \
    -i ../data/picrust2/03-EC_metagenome_out/pred_metagenome_contrib.tsv.gz \
    -o ../data/picrust2/04-pathways_out_per_seq \
    --per_sequence_contrib \
    --per_sequence_abun ../data/picrust2/03-EC_metagenome_out/seqtab_norm.tsv.gz \
    --per_sequence_function ../data/picrust2/02-EC_predicted.tsv.gz

Traceback (most recent call last):
  File "/home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/bin/pathway_pipeline.py", line 272, in <module>
    main()
  File "/home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/bin/pathway_pipeline.py", line 210, in main
    verbose=args.verbose)
  File "/home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/lib/python3.6/site-packages/picrust2/pathway_pipeline.py", line 355, in pathway_pipeline
    index=False)
  File "/home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/lib/python3.6/site-packages/pandas/core/generic.py", line 3228, in to_csv
    formatter.save()
  File "/home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/lib/python3.6/site-packages/pandas/io/formats/csvs.py", line 202, in save
    self._save()
  File "/home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/lib/python3.6/site-packages/pandas/io/formats/csvs.py", line 324, in _save
    self._save_chunk(start_i, end_i)
  File "/home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/lib/python3.6/site-package

In [14]:
! pathway_pipeline.py \
    -i ../data/picrust2/03-EC_metagenome_out/pred_metagenome_contrib.tsv.gz \
    -o ../data/picrust2/04-pathways_out

Traceback (most recent call last):
  File "/home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/lib/python3.6/site-packages/pandas/io/formats/csvs.py", line 202, in save
    self._save()
  File "/home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/lib/python3.6/site-packages/pandas/io/formats/csvs.py", line 324, in _save
    self._save_chunk(start_i, end_i)
  File "/home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/lib/python3.6/site-packages/pandas/io/formats/csvs.py", line 356, in _save_chunk
    libwriters.write_csv_rows(self.data, ix, self.nlevels, self.cols, self.writer)
  File "pandas/_libs/writers.pyx", line 69, in pandas._libs.writers.write_csv_rows
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/WIN.DTU.DK/matinnu/miniconda3/envs/picrust2/bin/pathway_pipeline

In [15]:
! pathway_pipeline.py \
    -i ../data/picrust2/03-EC_metagenome_out/pred_metagenome_unstrat.tsv.gz \
    -o ../data/picrust2/04-pathways_out \
    --intermediate ../data/picrust2/04-pathways_out/minpath_working

In [11]:
#! zless ../data/picrust2/04-pathways_out_per_seq/path_abun_contrib.tsv.gz

In [12]:
df_ec = pd.read_csv('test/q2-picrust2_output/ec_metagenome/ec.tsv', sep=('\t'), skiprows=[0], index_col=0)
df_p8 = df_ec[['P8-rep1', 'P8-rep2', 'P8-rep3', 'P8-rep4', 'P8-rep5']].sort_values('P8-rep2', ascending = False)
df_p8.loc[~(df_p8==0).all(axis=1)].head(10).style.background_gradient(cmap ='viridis')
! biom convert -i test/table-0.1/feature-table.biom -o test/table-0.1/feature-table.tsv --to-tsv
df_ec = pd.read_csv('test/table-0.1/feature-table.tsv', sep=('\t'), skiprows=[0], index_col=0)
#df_p8 = df_ec[['P8-rep1', 'P8-rep2', 'P8-rep3', 'P8-rep4', 'P8-rep5']].sort_values('P8-rep2', ascending = False)
df_p8.loc[~(df_p8==0).all(axis=1)].head(10).style.background_gradient(cmap ='viridis')

FileNotFoundError: [Errno 2] File b'test/q2-picrust2_output/ec_metagenome/ec.tsv' does not exist: b'test/q2-picrust2_output/ec_metagenome/ec.tsv'